<div align="right"> Massimo Nocentini,<br>Florence 2016</div>
<br>
<div align="center">
<b>abstract</b><br>
In this notebook we study two recurrence relations arising from the analysis of the `Quicksort` algorithm: numbers of checks and swaps are taken into account, in the average case. Such relations involve subterms where subscripts dependends on *one* dimension. They are a simple, but interesting, starting point to approach the general method of <b>unfolding</b>, an algorithmic/symbolical idea stretched further in other notebooks.
</div>
<br>
<div align="right">
    <img src="http://www.cerm.unifi.it/chianti/images/logo%20unifi_positivo.jpg" 
        alt="UniFI logo" style="width:20%;height:20%;">
</div>

In [2]:
%run "recurrences.py"
%run "start_session.py"

# Quicksort number of checks $c_{n}$, the average case

Define the recurrence relation as we did on paper, namely as an equation coupling an *indexable* symbol on both sides.

In [3]:
c = IndexedBase('c')
checks_recurrence = Eq(c[n]/(n+1), 2/(n+1) + c[n-1]/n)
checks_recurrence

 c[n]     2     c[n - 1]
───── = ───── + ────────
n + 1   n + 1      n    

Function `make_recurrence_spec` builds an object that denote the recurrence, formally and programmatically.

In [4]:
checks_recurrence_spec=make_recurrence_spec(recurrence_eq=checks_recurrence, indexed=c, index=make_index(n))

Function `do_unfolding_steps` allow us to perform *unfolding* or *unrolling* on occurrences of the inductively defined symbol. Doing $n$ steps of unfolding is the same to say to use the main recurrence defined above as a rewriting rule for occurrences, on the rhs, that pattern match with the one on the lhs. 

In [5]:
unfolded = do_unfolding_steps(checks_recurrence_spec, steps=4)

The following is the equation produced after 4 steps.

In [6]:
unfolded.recurrence_eq

 c[n]     2     c[n - 5]     2     2     2       2  
───── = ───── + ──────── + ───── + ─ + ───── + ─────
n + 1   n - 3    n - 4     n + 1   n   n - 1   n - 2

However, it is interesting to lock at the *history* of discovered terms in order to yield the previous result.

In [16]:
unfolded.terms_cache

⎧c[n - 1]  c[n - 2]   2  c[n - 4]    2     c[n - 5]  c[n - 3]    2     c[n - 4
⎨────────: ──────── + ─, ────────: ───── + ────────, ────────: ───── + ───────
⎩   n       n - 1     n   n - 3    n - 3    n - 4     n - 2    n - 2    n - 3 

]  c[n - 2]    2     c[n - 3]⎫
─, ────────: ───── + ────────⎬
    n - 1    n - 1    n - 2  ⎭

If desired, we can *base* the previous equation such that a new equation is produced containing the very first term of the sequence, in our case $c_{0}$.

In [7]:
with instantiate_eq(unfolded.recurrence_eq, {n:5}) as instantiated_eq: pass
instantiated_eq

c[5]          29
──── = c[0] + ──
 6            10

Using the previous idea, finding a value for $n$ that causes the very first item to appear, then we can substitute in the entire specification.

In [17]:
based_recurrence_spec = base_instantiation(unfolded, make_index(0), unary_subscripts_subsume_sols)
based_recurrence_spec

⎛c[5]          29             ⎧c[1]            c[2]  c[1]   2  c[3]  c[2]   1 
⎜──── = c[0] + ──, {n: 5}, c, ⎨────: c[0] + 1, ────: ──── + ─, ────: ──── + ─,
⎝ 6            10             ⎩ 2               3     2     3   4     3     2 

 c[4]  c[3]   2⎫⎞
 ────: ──── + ─⎬⎟
  5     4     5⎭⎠

If we attach a value to $c_{0}$ and we put it in the term cache, then it is possible to *fully* instantiate the spec.

In [18]:
boundary_conditions = {c[0]:Integer(0)} # `Integer` object is mandatory, not a vanilla `int`
with copy_recurrence_spec(based_recurrence_spec) as rec_spec_for_full_instantiation:
    rec_spec_for_full_instantiation.terms_cache.update(boundary_conditions)
    fully_instantiated_spec = repeated_instantiating(rec_spec_for_full_instantiation)
fully_instantiated_spec

⎛c[5]   29             ⎧         c[1]     c[2]       c[3]        c[4]  77⎫⎞
⎜──── = ──, {n: 5}, c, ⎨c[0]: 0, ────: 1, ────: 5/3, ────: 13/6, ────: ──⎬⎟
⎝ 6     10             ⎩          2        3          4           5    30⎭⎠

Moreover, if we want to skip all little steps and perform a *batch* study for a variable number of steps, then we provide an *higher-order* operator `ipython_latex`, which produces a nice representation for a set of unfoldings

In [34]:
ipython_latex(checks_recurrence_spec, times_range=range(10), 
              base_index=make_index(0), subsume_sols=unary_subscripts_subsume_sols)

<IPython.core.display.Latex object>

# Quicksort number of swaps $s_{n}$, the average case

In [20]:
s = IndexedBase('s')
swaps_recurrence = Eq(s[n]/(n+1),s[n-1]/n + (2*n-3)/(6*n*(n+1)))
swaps_recurrence

 s[n]   s[n - 1]     2⋅n - 3  
───── = ──────── + ───────────
n + 1      n       6⋅n⋅(n + 1)

In [24]:
swaps_recurrence_spec=make_recurrence_spec(recurrence_eq=swaps_recurrence, indexed=s, index=make_index(n))

In [25]:
unfolded = do_unfolding_steps(swaps_recurrence_spec, steps=4)

In [26]:
unfolded.recurrence_eq

 s[n]         2⋅n                2⋅n             2⋅n          2⋅n       s[n - 
───── = ──────────────── + ──────────────── + ────────── + ────────── + ──────
n + 1      2                  2                  2            2          n - 4
        6⋅n  - 30⋅n + 36   6⋅n  - 18⋅n + 12   6⋅n  + 6⋅n   6⋅n  - 6⋅n         

5]        2⋅n - 11                7                  9               5        
── + ───────────────── + - ──────────────── - ──────────────── - ────────── - 
     6⋅(n - 4)⋅(n - 3)        2                  2                  2         
                           6⋅n  - 18⋅n + 12   6⋅n  - 30⋅n + 36   6⋅n  - 6⋅n   

    3     
──────────
   2      
6⋅n  + 6⋅n

In [27]:
with instantiate_eq(unfolded.recurrence_eq, {n:5}) as instantiated_eq: pass
instantiated_eq

s[5]              
──── = s[0] + 1/15
 6                

In [28]:
unfolded.terms_cache

⎧s[n - 1]  s[n - 2]     2⋅n - 5    s[n - 4]  s[n - 5]        2⋅n - 11      s[n
⎨────────: ──────── + ───────────, ────────: ──────── + ─────────────────, ───
⎩   n       n - 1     6⋅n⋅(n - 1)   n - 3     n - 4     6⋅(n - 4)⋅(n - 3)   n 

 - 3]  s[n - 4]        2⋅n - 9       s[n - 2]  s[n - 3]        2⋅n - 7     ⎫
─────: ──────── + ─────────────────, ────────: ──────── + ─────────────────⎬
- 2     n - 3     6⋅(n - 3)⋅(n - 2)   n - 1     n - 2     6⋅(n - 2)⋅(n - 1)⎭

In [29]:
based_recurrence_spec = base_instantiation(unfolded, make_index(0), unary_subscripts_subsume_sols)
based_recurrence_spec

⎛s[5]                           ⎧s[1]               s[2]  s[1]   1   s[3]  s[2
⎜──── = s[0] + 1/15, {n: 5}, s, ⎨────: s[0] - 1/12, ────: ──── + ──, ────: ───
⎝ 6                             ⎩ 2                  3     2     36   4     3 

]   1   s[4]  s[3]   1 ⎫⎞
─ + ──, ────: ──── + ──⎬⎟
    24   5     4     24⎭⎠

In [31]:
boundary_conditions = {s[0]:Integer(0)} # `Integer` object is mandatory, not a vanilla `int`
with copy_recurrence_spec(based_recurrence_spec) as rec_spec_for_full_instantiation:
    rec_spec_for_full_instantiation.terms_cache.update(boundary_conditions)
    fully_instantiated_spec = repeated_instantiating(rec_spec_for_full_instantiation)
fully_instantiated_spec

⎛s[5]                    ⎧         s[1]         s[2]         s[3]         s[4]
⎜──── = 1/15, {n: 5}, s, ⎨s[0]: 0, ────: -1/12, ────: -1/18, ────: -1/72, ────
⎝ 6                      ⎩          2            3            4            5  

      ⎫⎞
: 1/36⎬⎟
      ⎭⎠

In [33]:
ipython_latex(swaps_recurrence_spec, times_range=range(10), 
              base_index=make_index(0), subsume_sols=unary_subscripts_subsume_sols)

<IPython.core.display.Latex object>

---

<a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-nc-sa/4.0/88x31.png" /></a><br />This work by <a xmlns:cc="http://creativecommons.org/ns#" href="https://github.com/massimo-nocentini/" property="cc:attributionName" rel="cc:attributionURL">Massimo Nocentini</a> is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/">Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International License</a>.<br />Based on a work at <a xmlns:dct="http://purl.org/dc/terms/" href="https://github.com/massimo-nocentini/recurrences-unfolding" rel="dct:source">https://github.com/massimo-nocentini/recurrences-unfolding</a>.